In [ ]:
# Importing necessary libraries for data processing, modeling, and visualization.
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os


'/Users/alan/anaconda3/bin/python'

In [ ]:
# Load the dataset
absolute_path = '/Users/alan/Downloads/NVDA.csv'
df = pd.read_csv(absolute_path)
# Convert the 'Date' column to datetime format for time series analysis
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Normalize the stock prices between 0 and 1 to ensure faster convergence during training
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))


In [ ]:
# Splitting the dataset: 80% for training.
training_data_len = int(np.ceil(len(scaled_data) * 0.8))

# Creating training dataset: 
# Use the past 60 days stock prices to predict the next day's price
x_train, y_train = [], []
for i in range(60, training_data_len):
    x_train.append(scaled_data[i-60:i, 0])
    y_train.append(scaled_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [ ]:
# Splitting the dataset: 80% for training.
training_data_len = int(np.ceil(len(scaled_data) * 0.8))

# Creating training dataset: 
# Use the past 60 days stock prices to predict the next day's price
x_train, y_train = [], []
for i in range(60, training_data_len):
    x_train.append(scaled_data[i-60:i, 0])
    y_train.append(scaled_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [ ]:
# Creating test dataset:
# Remaining 20% of the data with a rolling window of 60 days.
test_data = scaled_data[training_data_len - 60:, :]
x_test, y_test = [], df['Close'][training_data_len:].values
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Making predictions using the trained LSTM model
predictions = model.predict(x_test)
# Converting normalized predictions back to original scale
predictions = scaler.inverse_transform(predictions)


In [ ]:
# Calculating the Root Mean Squared Error to evaluate model's accuracy
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Visualizing the actual vs predicted stock prices to see how well our model performed
plt.figure(figsize=(15,7))
plt.plot(df.index[training_data_len:], y_test, label="Actual", color='blue')
plt.plot(df.index[training_data_len:], predictions, label="Predicted", color='red')
plt.title('Actual vs Predicted Prices')
plt.legend()
plt.show()


In [ ]:
# Predicting stock prices for upcoming dates:
# The model will use the most recent 60 days of data to forecast future prices.
future_dates = pd.date_range(start='2023-10-25', end='2023-11-07', freq='B')
future_data = scaled_data[-60:]
for i in range(len(future_dates)):
    x = future_data[-60:]
    x = x.reshape((1, 60, 1))
    pred = model.predict(x)
    future_data = np.append(future_data, pred)


In [ ]:
# Convert future predictions back to their original scale
future_predictions = scaler.inverse_transform(future_data[-len(future_dates):].reshape(-1,1))

# Store predictions in a DataFrame with corresponding dates
forecast_df = pd.DataFrame(data=future_predictions, index=future_dates, columns=['Predicted Close'])

# Save the predictions to a CSV file for further analysis or sharing
filename = "Implementation1.csv"
current_directory = os.getcwd()
full_path = os.path.join(current_directory, filename)
forecast_df.to_csv(full_path)

print("Predictions saved to Implementation1.csv!")
